# Dataset Preprocessing

## Import and read json dataset

In [68]:
import pandas as pd
import json

In [69]:
JSON_PATH = '/content/appraisals_dataset.json'

with open(JSON_PATH) as f:
    data = json.load(f)

print("Total appraisals:", len(data['appraisals']))

print("Top-level keys in one appraisal:")
print(list(data['appraisals'][0].keys()))

Total appraisals: 88
Top-level keys in one appraisal:
['orderID', 'subject', 'comps', 'properties']


In [70]:
#Find column names
example = data['appraisals'][0]

subject_dict    = example['subject']      # a dict
comps_list      = example['comps']        # list of dicts
properties_list = example['properties']   # list of dicts

print("Subject columns:   ", list(subject_dict.keys()))
print("Comps columns:     ", list(comps_list[0].keys()))
print("Properties columns:", list(properties_list[0].keys()))

Subject columns:    ['address', 'subject_city_province_zip', 'effective_date', 'municipality_district', 'site_dimensions', 'lot_size_sf', 'units_sq_ft', 'year_built', 'structure_type', 'roofing', 'effective_age', 'style', 'construction', 'remaining_economic_life', 'windows', 'basement', 'exterior_finish', 'basement_area', 'foundation_walls', 'flooring', 'plumbing_lines', 'heating', 'fuel_type', 'water_heater', 'cooling', 'room_count', 'num_beds', 'room_total', 'main_lvl_area', 'second_lvl_area', 'third_lvl_area', 'gla', 'subject_age', 'num_baths', 'condition']
Comps columns:      ['distance_to_subject', 'prop_type', 'stories', 'address', 'city_province', 'sale_date', 'sale_price', 'dom', 'location_similarity', 'lot_size', 'age', 'condition', 'gla', 'room_count', 'bed_count', 'bath_count', 'basement_finish', 'parking', 'neighborhood']
Properties columns: ['id', 'address', 'bedrooms', 'gla', 'city', 'province', 'postal_code', 'property_sub_type', 'structure_type', 'style', 'levels', 'roo

In [71]:
#Confirm all have the same column names
for i in range(0,5):
  example = data['appraisals'][i]

  subject_dict    = example['subject']      # a dict
  comps_list      = example['comps']        # list of dicts
  properties_list = example['properties']   # list of dicts

  print("Subject columns:   ", list(subject_dict.keys()))
  # print("Comps columns:     ", list(comps_list[0].keys()))
  # print("Properties columns:", list(properties_list[0].keys()))

Subject columns:    ['address', 'subject_city_province_zip', 'effective_date', 'municipality_district', 'site_dimensions', 'lot_size_sf', 'units_sq_ft', 'year_built', 'structure_type', 'roofing', 'effective_age', 'style', 'construction', 'remaining_economic_life', 'windows', 'basement', 'exterior_finish', 'basement_area', 'foundation_walls', 'flooring', 'plumbing_lines', 'heating', 'fuel_type', 'water_heater', 'cooling', 'room_count', 'num_beds', 'room_total', 'main_lvl_area', 'second_lvl_area', 'third_lvl_area', 'gla', 'subject_age', 'num_baths', 'condition']
Subject columns:    ['address', 'subject_city_province_zip', 'effective_date', 'municipality_district', 'site_dimensions', 'lot_size_sf', 'units_sq_ft', 'year_built', 'structure_type', 'roofing', 'effective_age', 'style', 'construction', 'remaining_economic_life', 'windows', 'basement', 'exterior_finish', 'basement_area', 'foundation_walls', 'flooring', 'plumbing_lines', 'heating', 'fuel_type', 'water_heater', 'cooling', 'room_co

## Create pandas dataframe with common column names

In [72]:
# 1.  Helper: build one row-dict for each section
def build_subject_row(oid, s):
    return {
        'order_id'              : oid,
        'address'               : s.get('address'),
        'city'                  : s.get('municipality_district'),          # raw → to-clean
        'province'              : s.get('subject_city_province_zip') or s.get('address'),
        'postal_code'           : s.get('subject_city_province_zip') or s.get('address'),
        'dom'                   : s.get('effective_date'),                 # calc later
        'effective_date'        : s.get('effective_date'),
        'structure_type'        : s.get('structure_type'),
        'age'                   : s.get('subject_age'),
        'gla'                   : s.get('gla'),
        'stories'               : s.get('style'),
        'bedrooms'              : s.get('num_beds'),
        'lot_size'              : f"{s.get('lot_size_sf', '')}, {s.get('site_dimensions', '')}",
        'effective_age'         : s.get('effective_age'),
        'remaining_economic_age': s.get('remaining_economic_life'),
        'basement'              : s.get('basement'),
        'basement_area'         : s.get('basement_area'),
        'heating'               : s.get('heating'),
        'cooling'               : s.get('cooling'),
        'room_count'            : s.get('room_count') or s.get('room_total'),
        'full_baths'            : s.get('num_baths'),   # raw “1:1” → parse later
        'half_baths'            : s.get('num_baths'),
        'condition'             : s.get('condition'),
    }

def build_comp_row(oid, c, rank):
    return {
        'order_id'            : oid,
        'comp_rank'           : rank,
        'distance_to_subject' : c.get('distance_to_subject'),
        'structure_type'      : c.get('prop_type'),
        'stories'             : c.get('stories'),
        'address'             : c.get('address'),
        'city'                : c.get('city_province'),        # split later
        'province'            : c.get('city_province'),
        'sale_date'           : c.get('sale_date'),
        'sale_price'          : c.get('sale_price'),
        'dom'                 : c.get('dom'),
        'location_similarity' : c.get('location_similarity'),
        'lot_size'            : c.get('lot_size'),
        'age'                 : c.get('age'),
        'condition'           : c.get('condition'),
        'gla'                 : c.get('gla'),
        'room_count'          : c.get('room_count'),
        'bedrooms'            : c.get('bed_count'),
        'full_baths'          : c.get('bath_count'),     # “2:0” → parse later
        'half_baths'          : c.get('bath_count'),
        'basement'            : c.get('basement_finish'),
        'parking'             : c.get('parking'),
        'neighborhood'        : c.get('neighborhood'),
    }

def build_property_row(oid, p):
    return {
        'order_id'    : oid,
        'id'          : p.get('id'),
        'address'     : p.get('address'),
        'bedrooms'    : p.get('bedrooms'),
        'gla'         : p.get('gla'),
        'city'        : p.get('city'),
        'province'    : p.get('province'),
        'postal_code' : p.get('postal_code'),
        'structure_type': p.get('structure_type') or p.get('property_sub_type'),
        'stories'     : p.get('style') or p.get('levels'),
        'room_count'  : p.get('room_count'),
        'full_baths'  : p.get('full_baths'),
        'half_baths'  : p.get('half_baths'),
        'lot_size'    : p.get('lot_size_sf'),
        'age'         : p.get('year_built'),        # derive later
        'year_built'  : p.get('year_built'),
        'basement'    : p.get('basement'),
        'heating'     : p.get('heating'),
        'cooling'     : p.get('cooling'),
        'sale_date'   : p.get('close_date'),
        'sale_price'  : p.get('close_price'),
        'public_remarks': p.get('public_remarks'),
        'latitude'    : p.get('latitude'),
        'longitude'   : p.get('longitude'),
    }
# ────────────────────────────────────────────────────────────────
# 2.  Iterate once, collect rows
subj_rows, comp_rows, prop_rows = [], [], []

for ap in data['appraisals']:
    oid = ap['orderID']
    subj_rows.append(build_subject_row(oid, ap['subject']))
    for i, c in enumerate(ap['comps'], 1):
        comp_rows.append(build_comp_row(oid, c, i))
    for p in ap['properties']:
        prop_rows.append(build_property_row(oid, p))

# ────────────────────────────────────────────────────────────────
# 3.  Convert to DataFrames  (values still raw—parsing comes later)
df_subjects   = pd.DataFrame(subj_rows)
df_comps      = pd.DataFrame(comp_rows)
df_properties = pd.DataFrame(prop_rows)

# Quick shape check
print(df_subjects.shape, df_comps.shape, df_properties.shape)

(88, 23) (264, 23) (9820, 24)


In [73]:
print(df_subjects.columns)
print(df_comps.columns)
print(df_properties.columns)

Index(['order_id', 'address', 'city', 'province', 'postal_code', 'dom',
       'effective_date', 'structure_type', 'age', 'gla', 'stories', 'bedrooms',
       'lot_size', 'effective_age', 'remaining_economic_age', 'basement',
       'basement_area', 'heating', 'cooling', 'room_count', 'full_baths',
       'half_baths', 'condition'],
      dtype='object')
Index(['order_id', 'comp_rank', 'distance_to_subject', 'structure_type',
       'stories', 'address', 'city', 'province', 'sale_date', 'sale_price',
       'dom', 'location_similarity', 'lot_size', 'age', 'condition', 'gla',
       'room_count', 'bedrooms', 'full_baths', 'half_baths', 'basement',
       'parking', 'neighborhood'],
      dtype='object')
Index(['order_id', 'id', 'address', 'bedrooms', 'gla', 'city', 'province',
       'postal_code', 'structure_type', 'stories', 'room_count', 'full_baths',
       'half_baths', 'lot_size', 'age', 'year_built', 'basement', 'heating',
       'cooling', 'sale_date', 'sale_price', 'public_rema

## Clean the values in the dataset into a common format

In [74]:
import re, numpy as np
from datetime import datetime

TODAY = pd.Timestamp('today').normalize()   # for DOM calculations
PC_RE = re.compile(r'[A-Za-z]\d[A-Za-z]\s?\d[A-Za-z]\d')

def clean_price(s):
    return pd.to_numeric(s.astype(str).str.replace(r'[^0-9.]', '', regex=True), errors='coerce')

def split_bath(col):
    full = col.astype(str).str.extract(r'(\d+):?')[0].astype(float)
    half = col.astype(str).str.extract(r':(\d+)')[0].astype(float)
    return full, half

def extract_city(text):
    if pd.isna(text): return np.nan
    return text.split(',')[0].strip() if ',' in text else text.strip()

def extract_province(text):
    if pd.isna(text): return np.nan
    parts = re.split(r'\s+', text.strip())
    return parts[-3] if len(parts) >= 3 else np.nan   # e.g. "Kingston ON K7M"

def extract_postal(text):
    m = PC_RE.search(str(text))
    return m.group(0).replace(' ', '').upper() if m else np.nan

In [75]:
# Parse subjects
# city, province, postal
df_subjects['city']        = df_subjects['city'].fillna(df_subjects['address']).apply(extract_city)
df_subjects['province']    = df_subjects['province'].apply(extract_province)
df_subjects['postal_code'] = df_subjects['postal_code'].apply(extract_postal)

# DOM = days since effective_date
df_subjects['dom'] = (TODAY - pd.to_datetime(df_subjects['dom'], errors='coerce')).dt.days

# numeric casts
df_subjects['age']              = pd.to_numeric(df_subjects['age'], errors='coerce')
df_subjects['gla']              = pd.to_numeric(df_subjects['gla'].astype(str).str.replace(r'[^0-9.]','',regex=True), errors='coerce')
df_subjects['bedrooms']         = pd.to_numeric(df_subjects['bedrooms'], errors='coerce')
df_subjects['basement_area']    = pd.to_numeric(df_subjects['basement_area'], errors='coerce')
df_subjects['room_count']       = pd.to_numeric(df_subjects['room_count'], errors='coerce')
df_subjects['effective_age']    = pd.to_numeric(df_subjects['effective_age'], errors='coerce')
df_subjects['remaining_economic_age'] = pd.to_numeric(df_subjects['remaining_economic_age'], errors='coerce')

# baths
df_subjects['full_baths'], df_subjects['half_baths'] = split_bath(df_subjects['full_baths'])

In [76]:
# distance, price, gla
df_comps['distance_to_subject'] = pd.to_numeric(
        df_comps['distance_to_subject'].astype(str).str.replace(r'[^0-9.]','',regex=True), errors='coerce')

df_comps['sale_price'] = clean_price(df_comps['sale_price'])
df_comps['gla']        = pd.to_numeric(df_comps['gla'].astype(str).str.replace(r'[^0-9.]','',regex=True), errors='coerce')

# city / province
df_comps[['city','province']] = df_comps['city'].str.extract(r'^(.*?)\s+([A-Z]{2})')

# DOM
df_comps['dom'] = pd.to_numeric(df_comps['dom'].str.replace(r'[^0-9]','',regex=True), errors='coerce')

# baths
df_comps['full_baths'], df_comps['half_baths'] = split_bath(df_comps['full_baths'])

# other numeric
df_comps['age']         = pd.to_numeric(df_comps['age'], errors='coerce')
df_comps['room_count']  = pd.to_numeric(df_comps['room_count'], errors='coerce')
df_comps['bedrooms']    = pd.to_numeric(df_comps['bedrooms'], errors='coerce')

# sale_date to datetime
df_comps['sale_date'] = pd.to_datetime(df_comps['sale_date'], errors='coerce')

In [77]:
# Parse properties
# price / date
df_properties['sale_price'] = clean_price(df_properties['sale_price'])
df_properties['sale_date']  = pd.to_datetime(df_properties['sale_date'], errors='coerce')

# numeric casts
for c in ['gla','lot_size','room_count','bedrooms','full_baths','half_baths']:
    df_properties[c] = pd.to_numeric(df_properties[c], errors='coerce')

# stories: prefer explicit style, else levels
df_properties['stories'] = df_properties['stories'].str.replace(r'\s+',' ',regex=True).str.strip()

# age from year_built
df_properties['year_built'] = pd.to_numeric(df_properties['year_built'], errors='coerce')
CURRENT_YEAR = 2025
df_properties['age'] = CURRENT_YEAR - df_properties['year_built']
df_properties['age'].fillna(df_properties['age'].median(), inplace=True)

# # age from year_built
# df_properties['age'] = pd.to_numeric(
#         df_properties['age'], errors='coerce')
# df_properties.loc[df_properties['age'].isna() & df_properties['year_built'].notna(), 'age'
#                  ] = TODAY.year - pd.to_numeric(df_properties['year_built'], errors='coerce')

# clean strings
df_properties['city']        = df_properties['city'].str.strip()
df_properties['province']    = df_properties['province'].str.strip()
df_properties['postal_code'] = df_properties['postal_code'].apply(extract_postal)

<ipython-input-77-6ac9433bedc6>:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_properties['age'].fillna(df_properties['age'].median(), inplace=True)


In [78]:
print(df_subjects.head())
print(df_comps.head())
print(df_properties.head())

  order_id                                  address  \
0  4762597  142-950 Oakview Ave Kingston ON K7M 6W8   
1  4762739           7180 207 HWY Halifax NS B0J2L0   
2  4763065                11 PAUL AVE Ayr ON N0B1E0   
3  4763086   102 Stonewalk Dr Kemptville ON K0G 1J0   
4  4763087    407 105 Dunbrack St Halifax NS B3M3G7   

                                               city      province postal_code  \
0                                          Kingston         "Twin         NaN   
1  Halifax Regional Municipality - West Chezzetcook  Chezzetcook,      B0J2L0   
2                        Township of North Dumfries           Ayr      N0B1E0   
3                                   North Grenville            ON      K0G1J0   
4           Halifax Regional Municipality - Halifax       Halifax      B3M3G7   

   dom effective_date structure_type   age     gla  ... effective_age  \
0   37    Apr/11/2025      Townhouse  49.0  1044.0  ...          25.0   
1   31    Apr/17/2025       Detached

## Remove duplicate datapoints

In [79]:
import unicodedata, string
from scipy.spatial import cKDTree

# common street abbreviations → canonical short form
_ABBR = {
    ' avenue ': ' ave ', ' drive ': ' dr ', ' street ': ' st ',
    ' road ': ' rd ', ' crescent ': ' cres ', ' court ': ' ct ',
    ' lane ': ' ln ', ' boulevard ': ' blvd ', ' place ': ' pl ',
}

def canon_addr(addr: str) -> str:
    if pd.isna(addr):
        return ''
    txt = unicodedata.normalize('NFD', addr).encode('ascii','ignore').decode().lower()
    txt = txt.translate(str.maketrans('', '', string.punctuation))
    for k, v in _ABBR.items():
        txt = txt.replace(k, v)
    return re.sub(r'\s+', ' ', txt).strip()

# add canonical key
df_properties['canonical_address'] = df_properties['address'].apply(canon_addr)

# build KD-tree for quick spatial look-ups (lat/long in radians)
coords_rad = np.radians(df_properties[['latitude','longitude']].values.astype(float))
tree = cKDTree(coords_rad)

# 10 meters in radians on Earth ≈ 10 / 6371000
EPS = 10 / 6_371_000

groups = {}                 # idx → group_id
current_gid = 0

for idx, (addr, lat, lon) in df_properties[['canonical_address','latitude','longitude']].iterrows():
    if pd.isna(lat) or pd.isna(lon):
        continue
    if idx in groups:       # already assigned
        continue
    # candidate idxs within 10 m
    near = tree.query_ball_point(np.radians([lat, lon]), EPS)
    # same canonical address filter
    dup_idxs = [i for i in near if df_properties.at[i,'canonical_address'] == addr]
    if len(dup_idxs) > 1:
        for i in dup_idxs:
            groups[i] = current_gid
        current_gid += 1

def pick_best(rows):
    # richness = non-null count
    richness = rows.notna().sum(axis=1)
    best = rows.loc[richness.idxmax()]
    # tie-break on sale_date
    ties = rows[richness == richness.max()]
    if len(ties) > 1 and 'sale_date' in rows.columns:
        best = ties.sort_values('sale_date', ascending=False).iloc[0]
    return best

keep_indices = set()
for gid in set(groups.values()):
    members = [i for i,g in groups.items() if g == gid]
    best_row = pick_best(df_properties.loc[members])
    keep_indices.add(best_row.name)

# build deduplicated DataFrame
keep_df   = df_properties.loc[list(keep_indices)]
drop_df   = df_properties.drop(index=list(groups.keys()))
dedup_properties = (
    pd.concat([keep_df, drop_df], ignore_index=True)
      .reset_index(drop=True)
)

print("Original rows :", len(df_properties))
print("After dedup   :", len(dedup_properties))

Original rows : 9820
After dedup   : 7258


In [80]:
print(dedup_properties.head())

  order_id      id                    address  bedrooms     gla      city  \
0  4762597     367        463 Conservatory Dr       3.0  1500.0  Kingston   
1  4762597     378             311 Janette St       3.0  1500.0  Kingston   
2  4762597  138739  786 HIGH GATE PARK Drive        3.0  1300.0  Kingston   
3  4776648   77521          494 Roosevelt Dr        3.0  1300.0  Kingston   
4  4776648   77694            393 Bernice Dr        4.0  2470.0  Kingston   

  province postal_code          structure_type              stories  ...  \
0  Ontario      K7M9C8                Detached  Brick, Vinyl Siding  ...   
1  Ontario      K7P0K8      Freehold Townhouse             2-Storey  ...   
2  Ontario      K7M5Z8  Detached, Sidesplit 3           Sidesplit 3  ...   
3  Ontario      K7M4B1                Detached      Bungalow-Raised  ...   
4  Ontario      K7M5X2                Detached             2-Storey  ...   

   year_built         basement      heating       cooling  sale_date  \
0       

In [81]:
df_properties = dedup_properties
print(df_properties.head())

  order_id      id                    address  bedrooms     gla      city  \
0  4762597     367        463 Conservatory Dr       3.0  1500.0  Kingston   
1  4762597     378             311 Janette St       3.0  1500.0  Kingston   
2  4762597  138739  786 HIGH GATE PARK Drive        3.0  1300.0  Kingston   
3  4776648   77521          494 Roosevelt Dr        3.0  1300.0  Kingston   
4  4776648   77694            393 Bernice Dr        4.0  2470.0  Kingston   

  province postal_code          structure_type              stories  ...  \
0  Ontario      K7M9C8                Detached  Brick, Vinyl Siding  ...   
1  Ontario      K7P0K8      Freehold Townhouse             2-Storey  ...   
2  Ontario      K7M5Z8  Detached, Sidesplit 3           Sidesplit 3  ...   
3  Ontario      K7M4B1                Detached      Bungalow-Raised  ...   
4  Ontario      K7M5X2                Detached             2-Storey  ...   

   year_built         basement      heating       cooling  sale_date  \
0       

## Handle missing values

In [82]:
# Helper:  show missing-value counts & percent for any frame
def missing_report(df, name):
    miss = df.isna().sum()
    pct  = (miss / len(df) * 100).round(1)
    report = (
        pd.DataFrame({'missing': miss, 'percent': pct})
          .loc[miss.gt(0)]                    # keep only cols with at least one NaN
          .sort_values('percent', ascending=False)
    )
    print(f"\n{name} — rows: {len(df)}  |  columns with NaNs: {len(report)}")
    return report

subj_missing   = missing_report(df_subjects,   'SUBJECTS')
comps_missing  = missing_report(df_comps,      'COMPS')
props_missing  = missing_report(df_properties, 'PROPERTIES')

# Display the top 10 most-missing columns in each DF
print("\n--- Top 10 subjects ---")
display(subj_missing.head(10))

print("\n--- Top 10 comps ---")
display(comps_missing.head(10))

print("\n--- Top 10 properties ---")
display(props_missing.head(10))


SUBJECTS — rows: 88  |  columns with NaNs: 12

COMPS — rows: 264  |  columns with NaNs: 10

PROPERTIES — rows: 7258  |  columns with NaNs: 15

--- Top 10 subjects ---


,missing,percent
basement_area,24,27.3
half_baths,22,25.0
age,16,18.2
effective_age,4,4.5
bedrooms,3,3.4
remaining_economic_age,3,3.4
postal_code,1,1.1
structure_type,1,1.1
heating,1,1.1
cooling,1,1.1



--- Top 10 comps ---


,missing,percent
age,110,41.7
half_baths,90,34.1
bedrooms,11,4.2
room_count,7,2.7
distance_to_subject,5,1.9
structure_type,3,1.1
dom,3,1.1
location_similarity,3,1.1
city,2,0.8
province,2,0.8



--- Top 10 properties ---


,missing,percent
half_baths,4373,60.3
lot_size,3543,48.8
year_built,2858,39.4
full_baths,2415,33.3
postal_code,186,2.6
stories,161,2.2
bedrooms,123,1.7
gla,120,1.7
room_count,103,1.4
heating,77,1.1


In [83]:
# # ─────  Missing‐Data Handling  ─────────────────────────────

# # thresholds
# ROW_DROP_THRESH = 0.10   # we IMPUTE up to this rate, not drop
# COL_DROP_THRESH = 0.50   # drop columns at or above

# # ─── 1. SUBJECTS ─────────────────────────────────────────────────────
# # (fill gla from basement_area first)
# if 'basement_area' in df_subjects:
#     df_subjects.loc[
#         df_subjects['gla'].isna() & df_subjects['basement_area'].notna(),
#         'gla'
#     ] = df_subjects['basement_area'] * 2

# subj_m = df_subjects.isna().mean()
# # a) drop cols ≥50%
# drop_cols = subj_m[subj_m >= COL_DROP_THRESH].index
# df_subjects = df_subjects.drop(columns=drop_cols)

# # b) impute cols 10–50%
# impute_cols = subj_m[(subj_m >= ROW_DROP_THRESH) & (subj_m < COL_DROP_THRESH)].index
# for c in impute_cols:
#     if pd.api.types.is_numeric_dtype(df_subjects[c]):
#         df_subjects[c] = df_subjects[c].fillna(df_subjects[c].median())
#     else:
#         df_subjects[c] = df_subjects[c].fillna(df_subjects[c].mode().iloc[0])

# # c) drop rows only on truly essential columns
# essential = ['address','effective_date','structure_type','gla','bedrooms']
# df_subjects = df_subjects.dropna(subset=essential).reset_index(drop=True)



# # ─── 2. COMPS ────────────────────────────────────────────────────────
# comp_m = df_comps.isna().mean()
# df_comps = df_comps.drop(columns=comp_m[comp_m >= COL_DROP_THRESH].index)

# # impute the mid-range by group or simple fill
# mid = comp_m[(comp_m >= ROW_DROP_THRESH) & (comp_m < COL_DROP_THRESH)].index
# for c in mid:
#     if c == 'age':
#         df_comps[c] = df_comps.groupby('structure_type')[c]\
#                               .transform(lambda s: s.fillna(s.median()))
#     elif c == 'half_baths':
#         df_comps[c] = df_comps[c].fillna(0)
#     elif pd.api.types.is_numeric_dtype(df_comps[c]):
#         df_comps[c] = df_comps[c].fillna(df_comps[c].median())
#     else:
#         df_comps[c] = df_comps[c].fillna(df_comps[c].mode().iloc[0])

# # drop rows only on a few must-have fields
# must_have = ['distance_to_subject','sale_price','sale_date']
# df_comps = df_comps.dropna(subset=must_have).reset_index(drop=True)



# # ─── 3. PROPERTIES ──────────────────────────────────────────────────
# prop_m = df_properties.isna().mean()
# df_properties = df_properties.drop(columns=prop_m[prop_m >= COL_DROP_THRESH].index)

# # impute mid-range
# mid = prop_m[(prop_m >= ROW_DROP_THRESH) & (prop_m < COL_DROP_THRESH)].index
# for c in mid:
#     if c == 'lot_size':
#         df_properties[c] = df_properties.groupby('structure_type')[c]\
#                                         .transform(lambda s: s.fillna(s.median()))
#     elif c == 'full_baths':
#         bins = pd.cut(df_properties['bedrooms'], [0,2,4,99], labels=['1-2','3-4','5+'])
#         df_properties[c] = df_properties.groupby(bins)[c]\
#                                         .transform(lambda s: s.fillna(s.median()))
#     elif pd.api.types.is_numeric_dtype(df_properties[c]):
#         df_properties[c] = df_properties[c].fillna(df_properties[c].median())
#     else:
#         df_properties[c] = df_properties[c].fillna(df_properties[c].mode().iloc[0])

# # drop rows only on key columns
# must_have = ['address','sale_date','sale_price']
# df_properties = df_properties.dropna(subset=must_have).reset_index(drop=True)



# # ─── 4. Quick sanity check ───────────────────────────────────────────
# print("Remaining SUBJECT cols:", len(df_subjects.columns))
# print("Remaining COMPS   cols:", len(df_comps.columns))
# print("Remaining PROPS  cols:", len(df_properties.columns))

# from pandas.api.types import is_numeric_dtype
# print("\nAny NaNs left?")
# print("SUBJECTS:", df_subjects.isna().sum().sum())
# print("COMPS   :", df_comps   .isna().sum().sum())
# print("PROPS   :", df_properties.isna().sum().sum())

In [84]:
# Helper:  show missing-value counts & percent for any frame
def missing_report(df, name):
    miss = df.isna().sum()
    pct  = (miss / len(df) * 100).round(1)
    report = (
        pd.DataFrame({'missing': miss, 'percent': pct})
          .loc[miss.gt(0)]                    # keep only cols with at least one NaN
          .sort_values('percent', ascending=False)
    )
    print(f"\n{name} — rows: {len(df)}  |  columns with NaNs: {len(report)}")
    return report

subj_missing   = missing_report(df_subjects,   'SUBJECTS')
comps_missing  = missing_report(df_comps,      'COMPS')
props_missing  = missing_report(df_properties, 'PROPERTIES')

# Display the top 10 most-missing columns in each DF
print("\n--- Top 10 subjects ---")
display(subj_missing.head(10))

print("\n--- Top 10 comps ---")
display(comps_missing.head(10))

print("\n--- Top 10 properties ---")
display(props_missing.head(10))


SUBJECTS — rows: 88  |  columns with NaNs: 12

COMPS — rows: 264  |  columns with NaNs: 10

PROPERTIES — rows: 7258  |  columns with NaNs: 15

--- Top 10 subjects ---


,missing,percent
basement_area,24,27.3
half_baths,22,25.0
age,16,18.2
effective_age,4,4.5
bedrooms,3,3.4
remaining_economic_age,3,3.4
postal_code,1,1.1
structure_type,1,1.1
heating,1,1.1
cooling,1,1.1



--- Top 10 comps ---


,missing,percent
age,110,41.7
half_baths,90,34.1
bedrooms,11,4.2
room_count,7,2.7
distance_to_subject,5,1.9
structure_type,3,1.1
dom,3,1.1
location_similarity,3,1.1
city,2,0.8
province,2,0.8



--- Top 10 properties ---


,missing,percent
half_baths,4373,60.3
lot_size,3543,48.8
year_built,2858,39.4
full_baths,2415,33.3
postal_code,186,2.6
stories,161,2.2
bedrooms,123,1.7
gla,120,1.7
room_count,103,1.4
heating,77,1.1


# Statistical Modeling

## XGBoost

In [85]:
NUMERIC_FEATS = [
    'gla',
    'full_baths',
    'age',
    'bedrooms',
    'room_count',
    # 'dom'
]
CATEG_FEATS = [
    'structure_type', 'stories'
]

FEATURES = NUMERIC_FEATS + CATEG_FEATS

# df_props = df_properties.merge(
#     df_subjects[['order_id','effective_date']],
#     on='order_id', how='left'
# )

# # ensure sale_date & effective_date are datetime
# df_props['sale_date']      = pd.to_datetime(df_props['sale_date'],      errors='coerce')
# df_props['effective_date'] = pd.to_datetime(df_props['effective_date'], errors='coerce')

# # 2) Filter to the 90-day window
# df_recent = df_props[
#     (df_props['sale_date'] >= df_props['effective_date'] - pd.Timedelta(days=90))
#   & (df_props['sale_date'] <= df_props['effective_date'])
# ].copy()

# df_recent = df_recent.merge(
#     df_properties[['order_id','id','canonical_address']],
#     on=['order_id','id'],    # or just on='order_id' if 'id' differs
#     how='left'
# )

df_properties.rename(columns={'close_price':'sale_price'}, inplace=True)

# 2️⃣  Build positives + negatives from the same pool (df_comps & df_properties)
#    — positives: the 3 true comps per order
df_pos = df_comps[df_comps['comp_rank'] <= 3].copy()
df_pos['is_selected'] = 1

# negatives: all other recent candidates
pos_keys = set(zip(
    df_pos['order_id'],
    df_pos['address'].str.lower().str.strip()
))

neg_rows = []
for oid, grp in df_properties.groupby('order_id'): #was df_recent
    for _, row in grp.iterrows():
        key = (oid, str(row['address']).lower().strip())
        if key in pos_keys:
            continue
        rec = {f: row.get(f, np.nan) for f in FEATURES}
        rec['is_selected'] = 0
        rec['order_id']    = oid
        neg_rows.append(rec)

df_negs = pd.DataFrame(neg_rows, columns=FEATURES + ['is_selected','order_id'])

df_all = pd.concat([
    df_pos[FEATURES + ['is_selected','order_id']],
    df_negs
], ignore_index=True)

print("Class balance:\n", df_all['is_selected'].value_counts())

# 3️⃣  Group-wise split: 80% train+val vs 20% test
from sklearn.model_selection import GroupShuffleSplit, train_test_split
X_full = df_all[FEATURES]
y_full = df_all['is_selected']
groups = df_all['order_id']

gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(gss.split(X_full, y_full, groups))

X_temp, X_test = X_full.iloc[train_idx], X_full.iloc[test_idx]
y_temp, y_test = y_full.iloc[train_idx], y_full.iloc[test_idx]

# 4️⃣  Split the 80% into train vs validation (stratify on label)
X_tr, X_val, y_tr, y_val = train_test_split(
    X_temp, y_temp,
    test_size=0.2,
    random_state=42,
    stratify=y_temp
)

# 5️⃣  Build & fit the Pipeline with early stopping
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
import xgboost as xgb

prep = ColumnTransformer([
    ('cat',  OneHotEncoder(handle_unknown='ignore'), CATEG_FEATS),
    ('num',  'passthrough', NUMERIC_FEATS)
])

clf = xgb.XGBClassifier(
    tree_method='hist',
    max_depth=6,
    learning_rate=0.1,
    n_estimators=500,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric=['logloss','auc'],
    early_stopping_rounds=30,
    missing=np.nan
)

# encode train/validation
X_tr_enc  = prep.fit_transform(X_tr)
X_val_enc = prep.transform(X_val)

clf.fit(
    X_tr_enc, y_tr,
    eval_set=[(X_tr_enc, y_tr), (X_val_enc, y_val)],
    verbose=False
)

model = Pipeline([('prep', prep), ('xgb', clf)])

# 6️⃣  Final evaluation on the untouched test set
from sklearn.metrics import roc_auc_score, precision_recall_curve, classification_report
import numpy as np

X_test_enc = prep.transform(X_test)
y_prob     = model.predict_proba(X_test)[:,1]

print("AUC :", roc_auc_score(y_test, y_prob).round(3))

prec, rec, thr = precision_recall_curve(y_test, y_prob)
f1_scores      = 2*prec*rec/(prec+rec+1e-9)
best_thr       = thr[np.argmax(f1_scores)]
print("Best F1 threshold:", round(best_thr,3))

y_pred = (y_prob >= best_thr).astype(int)
print("\nClassification report:\n",
      classification_report(y_test, y_pred, digits=3))


Class balance:
 is_selected
0    7238
1     264
Name: count, dtype: int64
AUC : 0.99
Best F1 threshold: 0.266

Classification report:
               precision    recall  f1-score   support

           0      0.995     0.989     0.992      1848
           1      0.692     0.833     0.756        54

    accuracy                          0.985      1902
   macro avg      0.844     0.911     0.874      1902
weighted avg      0.987     0.985     0.985      1902



### Test model output

In [104]:
# ▶️ 1. Which subjects are in the test split?
test_subjects = df_all.iloc[test_idx]['order_id'].unique()

for oid in test_subjects[:5]:
    # —— 1) Subject info ——
    subj = df_subjects[df_subjects['order_id'] == oid]
    if subj.empty:
        print(f"\nSubject {oid}: not found in df_subjects")
        continue
    subj = subj.iloc[0]
    cols = ['address','city','province','postal_code','gla','age','structure_type','bedrooms']
    have = [c for c in cols if c in df_subjects.columns]
    print(f"\n=== Subject {oid} ===")
    for c in have:
        print(f"  {c:15}: {subj[c]}")

    # —— 2) Expected (true) comps ——
    true_addrs = df_pos[df_pos['order_id']==oid]['address']\
                   .str.lower().str.strip().tolist()
    print("\nExpected (true) comps:")
    for a in true_addrs:
        print("  •", a)

    # —— 3) Candidate pool ——
    cand = df_properties[df_properties['order_id'] == oid].copy()
    print(f"\nFound {len(cand)} candidates")
    if cand.empty:
        continue

    # —— 4) Which true comps are missing? ——
    pool_addrs = cand['address'].str.lower().str.strip().tolist()
    missing = set(true_addrs) - set(pool_addrs)
    if missing:
        print("\nMissing true comps from candidate pool:")
        for a in missing:
            print("  ✘", a)
    else:
        print("\nAll true comps are present in pool.")

    # —— 5) Score & flag ——
    cand['score']         = model.predict_proba(cand[FEATURES])[:,1]
    cand['is_true_comp']  = cand['address']\
                              .str.lower().str.strip()\
                              .isin(true_addrs).astype(int)

    # —— 6) Show top-3 ——
    top3 = cand.nlargest(3, 'score')
    print("\nTop-3 recommendations:")
    display(
      top3[[
        'address','city','sale_price','gla','age','structure_type',
        'score','is_true_comp'
      ]].assign(score=lambda d: d['score'].round(3))
    )


=== Subject 4761446 ===
  address        : 5004 Fox Run Pl Kingston ON K7P0E4
  city           : Kingston
  province       : Kingston
  postal_code    : K7P0E4
  gla            : 1678.0
  age            : 15.0
  structure_type : Detached
  bedrooms       : 3.0

Expected (true) comps:
  • 868 roshan dr
  • 1249 carfa cres
  • 891 safari dr

Found 31 candidates

Missing true comps from candidate pool:
  ✘ 868 roshan dr
  ✘ 1249 carfa cres
  ✘ 891 safari dr

Top-3 recommendations:


,address,city,sale_price,gla,age,structure_type,score,is_true_comp
335,792 Safari Dr,Kingston,637000.0,1335.0,1985.0,Detached,0.357,0
333,1028 Bauder Cres,Kingston,542000.0,900.0,1985.0,Detached,0.335,0
334,1126 Dunham St,Kingston,935000.0,2700.0,21.0,Detached,0.064,0



=== Subject 4756192 ===
  address        : 2306-530 3 St SE Calgary AB T2G 2L8
  city           : Calgary
  province       : AB
  postal_code    : T2G2L8
  gla            : 1109.0
  age            : 2024.0
  structure_type : High Rise Apartment
  bedrooms       : 2.0

Expected (true) comps:
  • 1606-1111 10 st sw
  • 3205-930 6 ave sw
  • unit 2602 - 510 6 avenue se

Found 283 candidates

Missing true comps from candidate pool:
  ✘ 3205-930 6 ave sw
  ✘ 1606-1111 10 st sw
  ✘ unit 2602 - 510 6 avenue se

Top-3 recommendations:


,address,city,sale_price,gla,age,structure_type,score,is_true_comp
3405,1710 9 Street SE,Calgary,656500.0,1044.0,120.0,Detached,0.181,0
3399,1214 Salisbury Avenue SE,Calgary,1550000.0,1418.0,0.0,Detached,0.179,0
3380,806 23 Avenue SE,Calgary,410000.0,852.0,115.0,Detached,0.161,0



=== Subject 4758316 ===
  address        : 119 110 Auburn Meadows View SE Calgary, Alberta T3M 2M2
  city           : Calgary
  province       : AB,
  postal_code    : T3M2M2
  gla            : 789.0
  age            : 2017.0
  structure_type : Condominium
  bedrooms       : 2.0

Expected (true) comps:
  • 216-100 auburn meadows manor se
  • 411-25 auburn meadows ave se
  • 105-22 auburn bay link se

Found 257 candidates

Missing true comps from candidate pool:
  ✘ 411-25 auburn meadows ave se
  ✘ 216-100 auburn meadows manor se
  ✘ 105-22 auburn bay link se

Top-3 recommendations:


,address,city,sale_price,gla,age,structure_type,score,is_true_comp
4044,47 Bartlett Row SE,Calgary,667500.0,1936.0,0.0,Detached,0.442,0
4081,37 Bartlett Way SE,Calgary,665000.0,1881.0,0.0,Detached,0.442,0
4025,49 Bartlett Way SE,Calgary,679900.0,2090.0,0.0,Detached,0.432,0



=== Subject 4758529 ===
  address        : 915 140 SIMCOE ST Toronto ON M5H4E9
  city           : City of Toronto
  province       : Toronto
  postal_code    : M5H4E9
  gla            : 1176.0
  age            : 21.0
  structure_type : High Rise Apartment
  bedrooms       : 2.0

Expected (true) comps:
  • 904-71 simcoe st
  • 3103-126 simcoe st
  • 2805-361 front st w

Found 735 candidates

Missing true comps from candidate pool:
  ✘ 904-71 simcoe st
  ✘ 3103-126 simcoe st
  ✘ 2805-361 front st w

Top-3 recommendations:


,address,city,sale_price,gla,age,structure_type,score,is_true_comp
4318,Unit 1112 - 7 Grenville Street,Toronto,410000.0,250.0,21.0,"Condo Apartment, Bachelor/Studio",0.060,0
4097,Unit 806 - 8 Widmer Street,Toronto,445000.0,250.0,21.0,"Condo Apartment, Apartment",0.059,0
4098,Unit 3709 - 82 Dalhousie Street,Toronto,415000.0,250.0,21.0,"Condo Apartment, Apartment",0.059,0



=== Subject 4759260 ===
  address        : 60 MILL LAKE RD Number 2 Simms Settlement NS B0J1T0
  city           : LUNENBURG COUNTY - SIMMS SETTLEMENT
  province       : Settlement
  postal_code    : B0J1T0
  gla            : 864.0
  age            : 53.0
  structure_type : Detached
  bedrooms       : 3.0

Expected (true) comps:
  • 46 highway 3
  • 7730 highway 329
  • 43 summer lane

Found 7 candidates

Missing true comps from candidate pool:
  ✘ 43 summer lane
  ✘ 7730 highway 329
  ✘ 46 highway 3

Top-3 recommendations:


,address,city,sale_price,gla,age,structure_type,score,is_true_comp
4992,40 Foxwood Drive,Nova Scotia,417000.0,1620.0,53.0,Single Family,0.046,0
1002,331 Conrads Road,Nova Scotia,408800.0,1528.0,36.0,Single Family,0.042,0
1003,902 Highway 329,Nova Scotia,375000.0,1260.0,48.0,Single Family,0.042,0


### Manually insert missing comps into properties

In [106]:
for oid in test_subjects[:5]:
    subj = df_subjects[df_subjects['order_id'] == oid]
    if subj.empty:
        print(f"\nSubject {oid}: not found in df_subjects")
        continue
    subj = subj.iloc[0]
    cols = ['address','city','province','postal_code','gla','age','structure_type','bedrooms']
    have = [c for c in cols if c in df_subjects.columns]
    print(f"\n=== Subject {oid} ===")
    for c in have:
        print(f"  {c:15}: {subj[c]}")

    # —— 2) Expected (true) comps ——
    true_addrs = df_pos[df_pos['order_id']==oid]['address']\
                   .str.lower().str.strip().tolist()
    print("\nExpected (true) comps:")
    for a in true_addrs:
        print("  •", a)

    cand = df_properties[df_properties['order_id']==oid].copy()

    print(f"\nFound {len(cand)} candidates")

    # 1) Find which true comps aren’t already in cand
    true_df = df_pos[df_pos['order_id']==oid].copy()
    # normalize addresses for comparison
    cand_addrs = set(cand['address'].str.lower().str.strip())
    missing = true_df.loc[
        ~true_df['address'].str.lower().str.strip().isin(cand_addrs)
    ].copy()

    if not missing.empty:
        print(f"Adding {len(missing)} missing true comps into pool…")
        # 2) Rename the comp‐columns to match properties’ schema
        #    (only keep the FEATURES + the ones you need for display)
        mapping = {
          'sale_price': 'sale_price',   # already matches
          'sale_date':  'sale_date',    # if you need it
          'age':        'age',
          'gla':        'gla',
          'structure_type':'structure_type',
          'bed_count':  'bedrooms',
          'bath_count': 'full_baths',   # you may need to split half/full
          'half_baths': 'half_baths',
          # add any other mappings you need…
        }
        # first, make sure your df_pos has sale_price etc. as numerics
        # then:
        missing = missing.rename(columns=mapping)

        # 3) Ensure all FEATURES are present
        for f in FEATURES:
            if f not in missing.columns:
                missing[f] = np.nan

        # 4) Set their order_id so the model knows which subject they belong to
        missing['order_id'] = oid

        # 5) Append into cand
        cand = pd.concat([cand, missing], ignore_index=True, sort=False)

    else:
        print("\nAll true comps are present in pool.")

    # —— now score the augmented pool ——
    cand['score'] = model.predict_proba(cand[FEATURES])[:,1]
    cand['is_true_comp'] = cand['address'].str.lower().str.strip()\
                              .isin(true_df['address'].str.lower().str.strip())\
                              .astype(int)

    top3 = cand.nlargest(3, 'score')
    print("\nTop-3 with injected true comps:")
    display(
      top3[[
        'address','city','sale_price','gla','age','structure_type',
        'score','is_true_comp'
      ]].assign(score=lambda d: d['score'].round(3))
    )


=== Subject 4761446 ===
  address        : 5004 Fox Run Pl Kingston ON K7P0E4
  city           : Kingston
  province       : Kingston
  postal_code    : K7P0E4
  gla            : 1678.0
  age            : 15.0
  structure_type : Detached
  bedrooms       : 3.0

Expected (true) comps:
  • 868 roshan dr
  • 1249 carfa cres
  • 891 safari dr

Found 31 candidates
Adding 3 missing true comps into pool…

Top-3 with injected true comps:


,address,city,sale_price,gla,age,structure_type,score,is_true_comp
5,792 Safari Dr,Kingston,637000.0,1335.0,1985.0,Detached,0.357,0
3,1028 Bauder Cres,Kingston,542000.0,900.0,1985.0,Detached,0.335,0
32,1249 Carfa Cres,Kingston,1095000.0,1741.0,5.0,Detached,0.157,1



=== Subject 4756192 ===
  address        : 2306-530 3 St SE Calgary AB T2G 2L8
  city           : Calgary
  province       : AB
  postal_code    : T2G2L8
  gla            : 1109.0
  age            : 2024.0
  structure_type : High Rise Apartment
  bedrooms       : 2.0

Expected (true) comps:
  • 1606-1111 10 st sw
  • 3205-930 6 ave sw
  • unit 2602 - 510 6 avenue se

Found 283 candidates
Adding 3 missing true comps into pool…

Top-3 with injected true comps:


,address,city,sale_price,gla,age,structure_type,score,is_true_comp
283,1606-1111 10 St SW,Calgary,517500.0,965.00,2012.0,High Rise Apartment,0.570,1
284,3205-930 6 Ave SW,Calgary,574900.0,980.69,2017.0,High Rise Apartment,0.570,1
285,Unit 2602 - 510 6 Avenue SE,NaN,607500.0,1037.62,2016.0,High Rise Apartment,0.538,1



=== Subject 4758316 ===
  address        : 119 110 Auburn Meadows View SE Calgary, Alberta T3M 2M2
  city           : Calgary
  province       : AB,
  postal_code    : T3M2M2
  gla            : 789.0
  age            : 2017.0
  structure_type : Condominium
  bedrooms       : 2.0

Expected (true) comps:
  • 216-100 auburn meadows manor se
  • 411-25 auburn meadows ave se
  • 105-22 auburn bay link se

Found 257 candidates
Adding 3 missing true comps into pool…

Top-3 with injected true comps:


,address,city,sale_price,gla,age,structure_type,score,is_true_comp
257,216-100 Auburn Meadows Manor SE,Calgary,378000.0,934.0,2020.0,Condominium,0.601,1
211,47 Bartlett Row SE,Calgary,667500.0,1936.0,0.0,Detached,0.442,0
248,37 Bartlett Way SE,Calgary,665000.0,1881.0,0.0,Detached,0.442,0



=== Subject 4758529 ===
  address        : 915 140 SIMCOE ST Toronto ON M5H4E9
  city           : City of Toronto
  province       : Toronto
  postal_code    : M5H4E9
  gla            : 1176.0
  age            : 21.0
  structure_type : High Rise Apartment
  bedrooms       : 2.0

Expected (true) comps:
  • 904-71 simcoe st
  • 3103-126 simcoe st
  • 2805-361 front st w

Found 735 candidates
Adding 3 missing true comps into pool…

Top-3 with injected true comps:


,address,city,sale_price,gla,age,structure_type,score,is_true_comp
735,904-71 Simcoe St,Toronto,970000.0,1340.0,36.0,High Rise Apartment,0.266,1
736,3103-126 Simcoe St,Toronto,785000.0,853.0,12.0,High Rise Apartment,0.266,1
737,2805-361 Front St W,Toronto,920000.0,1043.0,23.0,High Rise Apartment,0.266,1



=== Subject 4759260 ===
  address        : 60 MILL LAKE RD Number 2 Simms Settlement NS B0J1T0
  city           : LUNENBURG COUNTY - SIMMS SETTLEMENT
  province       : Settlement
  postal_code    : B0J1T0
  gla            : 864.0
  age            : 53.0
  structure_type : Detached
  bedrooms       : 3.0

Expected (true) comps:
  • 46 highway 3
  • 7730 highway 329
  • 43 summer lane

Found 7 candidates
Adding 3 missing true comps into pool…

Top-3 with injected true comps:


,address,city,sale_price,gla,age,structure_type,score,is_true_comp
7,46 Highway 3,Hubbards,375000.0,1138.0,NaN,Detached,0.719,1
8,7730 Highway 329,East River Point,348500.0,1092.0,NaN,Detached,0.719,1
9,43 Summer Lane,Boutiliers Point,375000.0,889.0,NaN,Detached,0.662,1


# Explainability

## Integrate SHAP values

In [97]:
!pip install shap -q

import shap

In [105]:
for oid in test_subjects[:5]:
    subj = df_subjects[df_subjects['order_id'] == oid]
    if subj.empty:
        print(f"\nSubject {oid}: not found in df_subjects")
        continue
    subj = subj.iloc[0]
    cols = ['address','city','province','postal_code','gla','age','structure_type','bedrooms']
    have = [c for c in cols if c in df_subjects.columns]
    print(f"\n=== Subject {oid} ===")
    for c in have:
        print(f"  {c:15}: {subj[c]}")

    # —— 2) Expected (true) comps ——
    true_addrs = df_pos[df_pos['order_id']==oid]['address']\
                   .str.lower().str.strip().tolist()
    print("\nExpected (true) comps:")
    for a in true_addrs:
        print("  •", a)

    cand = df_properties[df_properties['order_id']==oid].copy()

    print(f"\nFound {len(cand)} candidates")

    # 1) Find which true comps aren’t already in cand
    true_df = df_pos[df_pos['order_id']==oid].copy()
    # normalize addresses for comparison
    cand_addrs = set(cand['address'].str.lower().str.strip())
    missing = true_df.loc[
        ~true_df['address'].str.lower().str.strip().isin(cand_addrs)
    ].copy()

    if not missing.empty:
        print(f"Adding {len(missing)} missing true comps into pool…")
        # 2) Rename the comp‐columns to match properties’ schema
        #    (only keep the FEATURES + the ones you need for display)
        mapping = {
          'sale_price': 'sale_price',   # already matches
          'sale_date':  'sale_date',    # if you need it
          'age':        'age',
          'gla':        'gla',
          'structure_type':'structure_type',
          'bed_count':  'bedrooms',
          'bath_count': 'full_baths',   # you may need to split half/full
          'half_baths': 'half_baths',
          # add any other mappings you need…
        }
        # first, make sure your df_pos has sale_price etc. as numerics
        # then:
        missing = missing.rename(columns=mapping)

        # 3) Ensure all FEATURES are present
        for f in FEATURES:
            if f not in missing.columns:
                missing[f] = np.nan

        # 4) Set their order_id so the model knows which subject they belong to
        missing['order_id'] = oid

        # 5) Append into cand
        cand = pd.concat([cand, missing], ignore_index=True, sort=False)

    else:
        print("\nAll true comps are present in pool.")

    # —— now score the augmented pool ——
    cand['score'] = model.predict_proba(cand[FEATURES])[:,1]
    cand['is_true_comp'] = cand['address'].str.lower().str.strip()\
                              .isin(true_df['address'].str.lower().str.strip())\
                              .astype(int)

    top3 = cand.nlargest(3, 'score')
    print("\nTop-3 with injected true comps:")
    display(
      top3[[
        'address','city','sale_price','gla','age','structure_type',
        'score','is_true_comp'
      ]].assign(score=lambda d: d['score'].round(3))
    )

    prep      = model.named_steps['prep']
    xgb_mod   = model.named_steps['xgb']

    X_top3    = top3[FEATURES]
    X_top3_enc = prep.transform(X_top3)
    feat_names = prep.get_feature_names_out()

    explainer  = shap.TreeExplainer(xgb_mod)
    shap_vals  = explainer.shap_values(X_top3_enc)

    for i in range(len(top3)):
      df_shap = (
          pd.DataFrame({
              'feature':    feat_names,
              'shap_value': shap_vals[i]
          })
          .assign(abs_shap=lambda d: d['shap_value'].abs())
          .nlargest(5, 'abs_shap')
          .reset_index(drop=True)
      )
    display(df_shap[['feature','shap_value']])


=== Subject 4761446 ===
  address        : 5004 Fox Run Pl Kingston ON K7P0E4
  city           : Kingston
  province       : Kingston
  postal_code    : K7P0E4
  gla            : 1678.0
  age            : 15.0
  structure_type : Detached
  bedrooms       : 3.0

Expected (true) comps:
  • 868 roshan dr
  • 1249 carfa cres
  • 891 safari dr

Found 31 candidates
Adding 3 missing true comps into pool…

Top-3 with injected true comps:


,address,city,sale_price,gla,age,structure_type,score,is_true_comp
5,792 Safari Dr,Kingston,637000.0,1335.0,1985.0,Detached,0.357,0
3,1028 Bauder Cres,Kingston,542000.0,900.0,1985.0,Detached,0.335,0
32,1249 Carfa Cres,Kingston,1095000.0,1741.0,5.0,Detached,0.157,1


,feature,shap_value
0,cat__structure_type_Detached,0.688696
1,num__room_count,0.289942
2,num__bedrooms,0.147264
3,num__full_baths,-0.031408
4,num__gla,-0.025374



=== Subject 4756192 ===
  address        : 2306-530 3 St SE Calgary AB T2G 2L8
  city           : Calgary
  province       : AB
  postal_code    : T2G2L8
  gla            : 1109.0
  age            : 2024.0
  structure_type : High Rise Apartment
  bedrooms       : 2.0

Expected (true) comps:
  • 1606-1111 10 st sw
  • 3205-930 6 ave sw
  • unit 2602 - 510 6 avenue se

Found 283 candidates
Adding 3 missing true comps into pool…

Top-3 with injected true comps:


,address,city,sale_price,gla,age,structure_type,score,is_true_comp
283,1606-1111 10 St SW,Calgary,517500.0,965.00,2012.0,High Rise Apartment,0.570,1
284,3205-930 6 Ave SW,Calgary,574900.0,980.69,2017.0,High Rise Apartment,0.570,1
285,Unit 2602 - 510 6 Avenue SE,NaN,607500.0,1037.62,2016.0,High Rise Apartment,0.538,1


...

,address,city,sale_price,gla,age,structure_type,score,is_true_comp
735,904-71 Simcoe St,Toronto,970000.0,1340.0,36.0,High Rise Apartment,0.266,1
736,3103-126 Simcoe St,Toronto,785000.0,853.0,12.0,High Rise Apartment,0.266,1
737,2805-361 Front St W,Toronto,920000.0,1043.0,23.0,High Rise Apartment,0.266,1


,feature,shap_value
0,cat__stories_1 Storey,1.923667
1,cat__structure_type_Detached,-0.139760
2,num__age,-0.134880
3,cat__stories_2 Storey,-0.088150
4,num__room_count,0.081895



=== Subject 4759260 ===
  address        : 60 MILL LAKE RD Number 2 Simms Settlement NS B0J1T0
  city           : LUNENBURG COUNTY - SIMMS SETTLEMENT
  province       : Settlement
  postal_code    : B0J1T0
  gla            : 864.0
  age            : 53.0
  structure_type : Detached
  bedrooms       : 3.0

Expected (true) comps:
  • 46 highway 3
  • 7730 highway 329
  • 43 summer lane

Found 7 candidates
Adding 3 missing true comps into pool…

Top-3 with injected true comps:


,address,city,sale_price,gla,age,structure_type,score,is_true_comp
7,46 Highway 3,Hubbards,375000.0,1138.0,NaN,Detached,0.719,1
8,7730 Highway 329,East River Point,348500.0,1092.0,NaN,Detached,0.719,1
9,43 Summer Lane,Boutiliers Point,375000.0,889.0,NaN,Detached,0.662,1


,feature,shap_value
0,num__age,1.509492
1,cat__stories_1 Storey,1.277422
2,cat__structure_type_Detached,0.297059
3,num__room_count,0.107224
4,cat__stories_2-Storey,0.105554


## Human Language Explanations using GPT-4

In [147]:
!pip install --upgrade openai python-dotenv -q

In [116]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [136]:
def llm_explain_comp(feature_imp_df):
    bullets = "\n".join(
        f"- {row['feature']}: {row['value']} (impact {row['shap_value']:+.3f})"
        for _, row in feature_imp_df.iterrows()
    )
    prompt = f"""
You are an expert real-estate appraisal assistant. Below are key features and their influence on this property’s suitability as a comparable:

{bullets}

Write a concise, 2–3 sentence explanation of why this property is a strong or weak comparable for the subject property.
"""
    resp = client.chat.completions.create(
        model="gpt-4.1",
        messages=[
            {"role": "system", "content": "Generate appraisal-style explanations."},
            {"role": "user",   "content": prompt}
        ],
        max_tokens=150,
        temperature=0.7
    )
    return resp.choices[0].message.content

In [137]:
prep    = model.named_steps['prep']
xgb_mod = model.named_steps['xgb']
explainer = shap.TreeExplainer(xgb_mod)
feat_names = prep.get_feature_names_out()

for oid in test_subjects[:5]:
    subj = df_subjects[df_subjects['order_id'] == oid]
    if subj.empty:
        print(f"\nSubject {oid}: not found in df_subjects")
        continue
    subj = subj.iloc[0]
    cols = ['address','city','province','postal_code','gla','age','structure_type','bedrooms']
    have = [c for c in cols if c in df_subjects.columns]
    print(f"\n=== Subject {oid} ===")
    for c in have:
        print(f"  {c:15}: {subj[c]}")

    # —— 2) Expected (true) comps ——
    true_addrs = df_pos[df_pos['order_id']==oid]['address']\
                   .str.lower().str.strip().tolist()
    print("\nExpected (true) comps:")
    for a in true_addrs:
        print("  •", a)

    cand = df_properties[df_properties['order_id']==oid].copy()

    print(f"\nFound {len(cand)} candidates")

    # 1) Find which true comps aren’t already in cand
    true_df = df_pos[df_pos['order_id']==oid].copy()
    # normalize addresses for comparison
    cand_addrs = set(cand['address'].str.lower().str.strip())
    missing = true_df.loc[
        ~true_df['address'].str.lower().str.strip().isin(cand_addrs)
    ].copy()

    if not missing.empty:
        print(f"Adding {len(missing)} missing true comps into pool…")
        # 2) Rename the comp‐columns to match properties’ schema
        #    (only keep the FEATURES + the ones you need for display)
        mapping = {
          'sale_price': 'sale_price',   # already matches
          'sale_date':  'sale_date',    # if you need it
          'age':        'age',
          'gla':        'gla',
          'structure_type':'structure_type',
          'bed_count':  'bedrooms',
          'bath_count': 'full_baths',   # you may need to split half/full
          'half_baths': 'half_baths',
          # add any other mappings you need…
        }
        # first, make sure your df_pos has sale_price etc. as numerics
        # then:
        missing = missing.rename(columns=mapping)

        # 3) Ensure all FEATURES are present
        for f in FEATURES:
            if f not in missing.columns:
                missing[f] = np.nan

        # 4) Set their order_id so the model knows which subject they belong to
        missing['order_id'] = oid

        # 5) Append into cand
        cand = pd.concat([cand, missing], ignore_index=True, sort=False)

    else:
        print("\nAll true comps are present in pool.")

    # —— now score the augmented pool ——
    cand['score'] = model.predict_proba(cand[FEATURES])[:,1]
    cand['is_true_comp'] = cand['address'].str.lower().str.strip()\
                              .isin(true_df['address'].str.lower().str.strip())\
                              .astype(int)

    top3 = cand.nlargest(3, 'score')
    print("\nTop-3 with injected true comps:")
    display(
      top3[[
        'address','city','sale_price','gla','age','structure_type',
        'score','is_true_comp'
      ]].assign(score=lambda d: d['score'].round(3))
    )

    for idx, row in top3.iterrows():
            # SHAP contribution
            X_feat = row[FEATURES].to_frame().T
            X_enc  = prep.transform(X_feat)
            sv     = explainer.shap_values(X_enc)[0]
            shap_df = pd.DataFrame({'feature': feat_names, 'shap_value': sv})
            shap_df['abs_shap'] = shap_df['shap_value'].abs()
            top5 = shap_df.nlargest(5,'abs_shap')[['feature','shap_value']].reset_index(drop=True)

            # attach original values
            values = []
            for f in top5['feature']:
                typ, col, *rest = f.split('__')
                if typ == 'num':
                    values.append(X_feat.iloc[0][col])
                else:
                    values.append(rest[0] if rest else None)
            top5['value'] = values

            # call LLM
            explanation = llm_explain_comp(top5)
            print(f"\n💬 Explanation for Comp {idx+1}: {row['address']}\n{explanation}\n")


=== Subject 4761446 ===
  address        : 5004 Fox Run Pl Kingston ON K7P0E4
  city           : Kingston
  province       : Kingston
  postal_code    : K7P0E4
  gla            : 1678.0
  age            : 15.0
  structure_type : Detached
  bedrooms       : 3.0

Expected (true) comps:
  • 868 roshan dr
  • 1249 carfa cres
  • 891 safari dr

Found 31 candidates
Adding 3 missing true comps into pool…

Top-3 with injected true comps:


,address,city,sale_price,gla,age,structure_type,score,is_true_comp
5,792 Safari Dr,Kingston,637000.0,1335.0,1985.0,Detached,0.357,0
3,1028 Bauder Cres,Kingston,542000.0,900.0,1985.0,Detached,0.335,0
32,1249 Carfa Cres,Kingston,1095000.0,1741.0,5.0,Detached,0.157,1



💬 Explanation for Comp 6: 792 Safari Dr 
This property is a strong comparable for the subject property due to its similar year of construction and structure characteristics, which positively impact its suitability. While its slightly smaller gross living area has a minor negative effect, the ample room count and two-storey configuration further enhance its comparability.


💬 Explanation for Comp 4: 1028 Bauder Cres
This property is a strong comparable for the subject property due to its construction year (1985), which positively impacts suitability (+1.779), indicating similar age and likely condition. Its detached structure type (+0.264) and 2-storey layout (+0.126) further strengthen comparability, while only minor negative adjustments are needed for slightly lower gross living area (-0.091) and room count (-0.089). Overall, the positive attributes outweigh the minor differences, making it a solid comparable choice.


💬 Explanation for Comp 33: 1249 Carfa Cres
This property is a str

,address,city,sale_price,gla,age,structure_type,score,is_true_comp
283,1606-1111 10 St SW,Calgary,517500.0,965.00,2012.0,High Rise Apartment,0.570,1
284,3205-930 6 Ave SW,Calgary,574900.0,980.69,2017.0,High Rise Apartment,0.570,1
285,Unit 2602 - 510 6 Avenue SE,NaN,607500.0,1037.62,2016.0,High Rise Apartment,0.538,1



💬 Explanation for Comp 284: 1606-1111 10 St SW
This property is a strong comparable due to its relatively recent year of construction (2012) and favorable room count, both of which closely align with the subject property’s characteristics. Additionally, its single-storey configuration further enhances its suitability as a comparable, while the detached structure type has a minimal negative impact. Overall, the positive influences outweigh the negatives, making this a strong match.


💬 Explanation for Comp 285: 3205-930 6 Ave SW
This property is a strong comparable due to its relatively recent construction year (2017), which closely aligns with the subject’s likely age and adds significant relevance (+1.474 impact). Additionally, its 6-room count and one-storey configuration further enhance comparability, making it a suitable benchmark for valuation. However, the detached structure type slightly detracts from its overall suitability.


💬 Explanation for Comp 286: Unit 2602 - 510 6 Aven

...


💬 Explanation for Comp 258: 216-100 Auburn Meadows Manor SE
This property is a strong comparable for the subject property due to its recent construction year (2020), which significantly aligns with modern standards and contributes positively to its suitability (+1.511 impact). Additionally, its likely single-storey layout (+1.471 impact) and the presence of two full bathrooms (+0.092 impact) further enhance comparability, despite a minor negative impact from being detached (-0.137).


💬 Explanation for Comp 212: 47 Bartlett Row SE 
This property is a strong comparable for the subject property primarily due to its new construction (age 0.0, significant positive impact) and similar two-storey, detached structure, both of which closely align with typical subject property features. While the slightly higher room count has a minor negative impact, the comparable offers a similar number of full baths, further supporting its suitability as a match.


💬 Explanation for Comp 249: 37 Bartlett W

,address,city,sale_price,gla,age,structure_type,score,is_true_comp
735,904-71 Simcoe St,Toronto,970000.0,1340.0,36.0,High Rise Apartment,0.266,1
736,3103-126 Simcoe St,Toronto,785000.0,853.0,12.0,High Rise Apartment,0.266,1
737,2805-361 Front St W,Toronto,920000.0,1043.0,23.0,High Rise Apartment,0.266,1



💬 Explanation for Comp 736: 904-71 Simcoe St
This property is a strong comparable due to its one-storey layout, which significantly enhances its suitability (+1.928 impact). While its detached structure and age slightly detract from its comparability, the overall positive influence of the layout and adequate room count make it a strong match for the subject property.


💬 Explanation for Comp 737: 3103-126 Simcoe St
This property is a strong comparable due to its single-storey configuration, which significantly aligns with the subject property (+1.935 impact). While it is slightly older (-0.133 impact) and detached (-0.142 impact), the overall positive influence from its layout and room count (+0.082 impact) makes it a suitable match for comparison purposes.


💬 Explanation for Comp 738: 2805-361 Front St W
This property is a relatively strong comparable due to its one-storey configuration, which significantly aligns with the subject property (+1.924 impact), and a similar room count (

,address,city,sale_price,gla,age,structure_type,score,is_true_comp
7,46 Highway 3,Hubbards,375000.0,1138.0,NaN,Detached,0.719,1
8,7730 Highway 329,East River Point,348500.0,1092.0,NaN,Detached,0.719,1
9,43 Summer Lane,Boutiliers Point,375000.0,889.0,NaN,Detached,0.662,1



💬 Explanation for Comp 8: 46 Highway 3
This property is a strong comparable for the subject property due to its positive alignment with several key features, including overall property age, one-storey layout, and detached structure, each contributing notably to its suitability. Additionally, the room count of 6 closely matches typical market expectations, further supporting its use as a comparable. Overall, these factors indicate a high degree of similarity with the subject property.


💬 Explanation for Comp 9: 7730 Highway 329
This property is a strong comparable for the subject property due to its significant positive alignment in age and being a 1-storey, detached home, which closely matches typical subject property characteristics. Additionally, its 6-room count further supports its suitability, making it a relevant and reliable comparison in the appraisal analysis.


💬 Explanation for Comp 10: 43 Summer Lane
This property is a strong comparable due to its significant positive ali

# Self-Improving System